In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import VGG19,Xception
from tensorflow.keras.layers import Input,Flatten,Dense,BatchNormalization,Activation,Dropout,GlobalAveragePooling2D,MaxPooling2D,RandomFlip,RandomZoom,RandomRotation
from keras.datasets import cifar10

In [2]:
# (X_train, y_train), (X_test, y_test) = cifar10.load_data()
# 
# y_train=to_categorical(y_train)
# y_test=to_categorical(y_test)

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

train_filter = y_train.flatten() < 4
test_filter = y_test.flatten() < 4

# Zastosuj filtry, aby wybrać odpowiednie dane
X_train = X_train[train_filter]
y_train = y_train[train_filter]
X_test = X_test[test_filter]
y_test = y_test[test_filter]

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [10]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3), classes=y_train.shape[1])
base_model.trainable = False

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model

inputs = tf.keras.Input(shape=(32, 32, 3))
x = Lambda(lambda image: tf.image.resize(image, (224, 224)))(inputs)
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(4, activation='softmax')(x)
model = Model(inputs, outputs)

In [16]:
# inputs = tf.keras.Input(shape=(32, 32, 3))
# x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (224,224)))(inputs)
# x = tf.keras.applications.xception.preprocess_input(x)
# x = base_model(x, training=False)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Dropout(0.3)(x)
# outputs = tf.keras.layers.Dense(10, activation=('softmax'))(x)
# model = tf.keras.Model(inputs, outputs)

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
epochs = 10
batch_size = 32
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/10
  4/625 [..............................] - ETA: 19:06 - loss: 1.4610 - accuracy: 0.2578

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
# model.save('xception_cifar.h5')

In [ ]:
model.save('xception_cifar_4_categories.h5')